<a href="https://colab.research.google.com/github/percivalyan/GenerateDummyDataKaryawan/blob/main/SawahFuzzy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl
import pandas as pd
import time

# Input variables (factors affecting soil fertility)
pH = ctrl.Antecedent(np.arange(0, 11, 1), 'pH')
nitrogen = ctrl.Antecedent(np.arange(0, 101, 1), 'nitrogen')
texture = ctrl.Antecedent(np.arange(0, 101, 1), 'texture')

# Output variable (soil fertility)
fertility = ctrl.Consequent(np.arange(0, 101, 1), 'fertility')

# Membership functions for pH
pH['acidic'] = fuzz.trimf(pH.universe, [0, 0, 6])
pH['neutral'] = fuzz.trimf(pH.universe, [4, 6, 8])
pH['alkaline'] = fuzz.trimf(pH.universe, [6, 10, 10])

# Membership functions for nitrogen
nitrogen['low'] = fuzz.trimf(nitrogen.universe, [0, 0, 50])
nitrogen['medium'] = fuzz.trimf(nitrogen.universe, [25, 50, 75])
nitrogen['high'] = fuzz.trimf(nitrogen.universe, [50, 100, 100])

# Membership functions for texture
texture['clay'] = fuzz.trimf(texture.universe, [0, 0, 50])
texture['silt'] = fuzz.trimf(texture.universe, [25, 50, 75])
texture['sand'] = fuzz.trimf(texture.universe, [50, 100, 100])

# Membership functions for fertility
fertility['poor'] = fuzz.trimf(fertility.universe, [0, 0, 50])
fertility['average'] = fuzz.trimf(fertility.universe, [25, 50, 75])
fertility['high'] = fuzz.trimf(fertility.universe, [50, 100, 100])

# Rule base
rule1 = ctrl.Rule(pH['acidic'] & nitrogen['low'] & texture['clay'], fertility['poor'])
rule2 = ctrl.Rule(pH['alkaline'] & nitrogen['high'] & texture['sand'], fertility['high'])
rule3 = ctrl.Rule(pH['neutral'] & nitrogen['medium'] & texture['silt'], fertility['average'])

# Control system
fertility_ctrl = ctrl.ControlSystem([rule1, rule2, rule3])

# Define data for 10 sawahs
sawah_data = [
    {'pH': 5.2, 'nitrogen': 55, 'texture': 50},
    {'pH': 6.0, 'nitrogen': 65, 'texture': 45},
    {'pH': 4.5, 'nitrogen': 35, 'texture': 60},
    {'pH': 6.3, 'nitrogen': 48, 'texture': 40},
    {'pH': 5.7, 'nitrogen': 52, 'texture': 55},
    {'pH': 7.0, 'nitrogen': 70, 'texture': 30},
    {'pH': 4.8, 'nitrogen': 42, 'texture': 65},
    {'pH': 6.4, 'nitrogen': 58, 'texture': 35},
    {'pH': 5.3, 'nitrogen': 50, 'texture': 70},
    {'pH': 7.2, 'nitrogen': 85, 'texture': 55}
]


# Initialize empty DataFrame to store results
results = []

# Simulate for each sawah
for i, data in enumerate(sawah_data, 1):
    fertility_evaluator = ctrl.ControlSystemSimulation(fertility_ctrl)

    # Example input values for each sawah
    fertility_evaluator.input['pH'] = data['pH']
    fertility_evaluator.input['nitrogen'] = data['nitrogen']
    fertility_evaluator.input['texture'] = data['texture']

    # Compute
    start_time = time.time()
    try:
        fertility_evaluator.compute()
        elapsed_time = time.time() - start_time

        # Get output fertility value
        fertility_value = fertility_evaluator.output['fertility']

        # Store results in a dictionary
        results.append({
            'Sawah': f'Sawah-{i}',
            'pH': data['pH'],
            'Nitrogen': data['nitrogen'],
            'Texture': data['texture'],
            'Fertility': fertility_value,
            'Elapsed Time (sec)': elapsed_time
        })
    except ValueError as e:
        print(f"Error computing for Sawah-{i}: {e}")
        continue

# Convert results to DataFrame
df_results = pd.DataFrame(results)

# Display the results
print(df_results)


      Sawah   pH  Nitrogen  Texture  Fertility  Elapsed Time (sec)
0   Sawah-1  5.2        55       50   50.00000            0.002272
1   Sawah-2  6.0        65       45   50.00000            0.000533
2   Sawah-3  4.5        35       60   50.00000            0.000497
3   Sawah-4  6.3        48       40   50.00000            0.000525
4   Sawah-5  5.7        52       55   50.00000            0.000558
5   Sawah-6  7.0        70       30   50.00000            0.000494
6   Sawah-7  4.8        42       65   50.00000            0.000481
7   Sawah-8  6.4        58       35   50.00000            0.000493
8   Sawah-9  5.3        50       70   50.00000            0.000510
9  Sawah-10  7.2        85       55   76.22807            0.000603
